In [1]:
pip install boto3==1.34.66

  Using cached boto3-1.34.66-py3-none-any.whl.metadata (6.6 kB)
Using cached boto3-1.34.66-py3-none-any.whl (139 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 47.0 MB/s eta 0:00:0000:010:01m
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.32
    Uninstalling botocore-1.34.32:
      Successfully uninstalled botocore-1.34.32
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.32
    Uninstalling boto3-1.34.32:
      Successfully uninstalled boto3-1.34.32
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.32 requires botocore==1.34.32, but you have botocore 1.34.76 which is incompatible.
awscli 1.32.32 requires colorama<0.4.5,>=0.2.5, but you have colorama 0.4.6 which is incompatible.
sagemaker 2.206.0 requires urllib3<1.27, but you have urllib3 2.0.7 which is incompatible.
Note: you may

In [5]:
import boto3
import json
import base64
import time
import re
import os
from botocore.exceptions import ClientError
import logging
import redis
from datetime import datetime , date
from insights import construct_call_conversation , enrollment_prompt_generator , load_claude2 , sns_data_postprocessing  
from insights import sns_publisher , get_insights_prompt , get_enrollment_prompt , insights_prompt_generator

today_date = date.today()

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name= os.environ.get("Region") #"us-east-1",
)
 
bedrock = boto3.client(
    service_name='bedrock', 
    region_name= os.environ.get("Region")  #'us-east-1'
)
 
bedrock_agent_runtime = boto3.client(
    service_name="bedrock-agent-runtime",
    region_name= os.environ.get("Region") #"us-east-1",
)
# Initialize Redis client

redis_host = 'ch-agent-assist-redis-cluster-disabled.tjyhst.ng.0001.use1.cache.amazonaws.com'
#os.environ.get("RedisReadUrl") # 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com'

redis_port = 6379 # Default Redis port is 6379

redis_password = None  # None if no password

redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
 
prompt_bucket = 'ch-agent-assist-prompt-library-bucket'

file_key = 'prompts_library.json'

#AWS SDK for importing the KB
def retrieveAndGenerate(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'WWGXWYHCZE',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2'
                }
            }
        )

def retrieveAndGenerate_new(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'HZ2VTAKFFD',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
                }
            }
        )


def retrieveAndGenerate_insurance_pharmacy(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'TZYCKWDKBC',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
                }
            }
        )

#Checking response
#kb_response = retrieveAndGenerate("What are the different type of insurance types", "WWGXWYHCZE")["output"]["text"]
#print(kb_response)
#kb_response_payer_pharmacy_list = retrieveAndGenerate("What are the different type of insurance providers? and for each of insurance providers, provide the time to fill for all of the specialty pharamcies, provide the response in a structure JSON format", "WWGXWYHCZE")["output"]["text"]
#print(kb_response_payer_pharmacy_list) 

#Passing the payer vs pharmacy stats as part of the context
 
entity_dict_v1 = {
    "patientFirstName": "",
    "patientMiddleInitial": "",
    "patientLastName": "",
    "dob": "",
    "age": "",
    "gender": "",
    "preferredLanguage": "",
    "streetAddress": "",
    "streetName": "",
    "city": "",
    "state": "",
    "zipCode": "",
    "email": "",
    "phone": "",
    "phoneType": "",
    "priorTherapy": "",
    "diagnosisName": "",
    "diagnosisIcd10Code": "",
    "dateOfDiagnosis": "",
    "insuranceType": "",
    "insuranceProvider": "",
    "insuranceId": "",
    "planType": "",
    "effectiveDate": "",
    "expiryDate": "",
    "recordType": "",
    "rxBin": "",
    "rxGroup": "",
    "rxPcn": "",
    "cardHolderRelationshipWithThePatient": "",
    "cardHolderName": "",
    "cardHolderDob": "",
    "prescriberName": "",
    "specialty": "",
    "address": "",
    "facilityName": "",
    "pharmacyName": "",
    "spPhone": "",
    "spFax": ""
}

entity_list = list(entity_dict_v1.keys())

entities = ""

for val in entity_list:
    entities += val + "," 

entities = entities[:len(entities)-1]
# Lambda handler to intgerate with AWS

def handler(event,context):
    start_timestamp = datetime.now()
    print(f'insights_data: handler: START: {datetime.now()}: {str(event)}')
    contact_id = json.loads(event["Records"][0]['body'])["streamConnectionId"] #event.get('conversation_id')
    #print(key_to_read)
    # Check if the key exists and read the value from Redis
    if contact_id and redis_client.exists(contact_id):
        value = redis_client.get(contact_id)
        print(value) #.split("contactId"))
    else:
        print('Key not found')
    #final_transcript = ""
    #for i in range(len(event['Records'])):
     #   final_transcript += "\n" + data_preprocessing(event['Records'][i])
    final_transcript = construct_call_conversation(value)
    #print(final_transcript)
    
    #sample_transcript = "Customer : Hello , I'm Mark."
    #prompt_enrollment = get_enrollment_prompt(prompt_bucket,file_key,'entity_extraction','enrollment_form_claude',final_transcript)
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities,today_date)
    #prompt_enrollment = enrollment_prompt_generator(sample_transcript,entities)
    entity_extraction_model_invoke_timestamp = datetime.now()
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    entity_extraction_model_invoke_after_timestamp = datetime.now()
    print(enrollment_data)
    #json_data = data_postprocessing(enrollment_data)
    json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
    match = json_pattern.search(enrollment_data)
    extracted_json = match.group(0)
    enrollment_json_object = json.loads(extracted_json)
    print("Printing all the keys",list(enrollment_json_object.keys()))
    enrollment_keys = list(enrollment_json_object.keys())
    if "insuranceProvider" in enrollment_json_object.keys() and "pharmacyName" in enrollment_json_object.keys() and enrollment_json_object["insuranceProvider"] != "" and enrollment_json_object["pharmacyName"] != "":
        redis_key = str(json.loads(event["Records"][0]['body'])["streamConnectionId"]) + '::insights_data'
        if redis_client.exists(redis_key):
            print(f'redis_key:{redis_key}')
            insights_redis_data_value = redis_client.get(redis_key)
            print(f'insights_redis_data_value:{insights_redis_data_value}')
            insights_redis_data = eval(insights_redis_data_value)
            print(f'insights_redis_data : {insights_redis_data}')
        #redis_client.set(redis_key,str([{"insuranceProvider":'United Health','pharmacyName':'CVS','insights':'Insights'}]))
        #redis_client.set(redis_key,str([]))
        #insights_redis_data_value = redis_client.get(redis_key)
        #print(f'insights_redis_data_value:{insights_redis_data_value}')
        #insights_redis_data = eval(insights_redis_data_value)
        #print(f'insights_redis_data : {insights_redis_data}')
        if len(insights_redis_data) == 0:
            insurance_provider = enrollment_json_object["insuranceProvider"]
            pharmacyName = enrollment_json_object["pharmacyName"]
            #insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
            insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
            insights_model_invoke_timestamp = datetime.now()
            insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
            insights_model_invoke_after_timestamp = datetime.now()
            print("Generating Insights successfully", insights)
            sns_data = sns_data_postprocessing(event,insights)
            print("sns_data : ",sns_data)
            sns_publisher(sns_data)
            insights_redis_data.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
            print(f'new_value : {insights_redis_data}')
            redis_client.set(redis_key,str(insights_redis_data))
            return {'statusCode':200,'body':'Data Sent to SNS Sucessfully.'}
          
        for val in insights_redis_data:
            if val['insuranceProvider'] == enrollment_json_object['insuranceProvider'] and val['pharmacyName'] == enrollment_json_object['pharmacyName']:
                print(f'insights_redis_data:{insights_redis_data}')
                return {"statusCode" : 200 , "body" : "Generated Insight already present in the database."}
                print('Generated Insight already present in the database.')
            #pass
            else:
                print('Have come to else...')
                insurance_provider = enrollment_json_object["insuranceProvider"]
                pharmacyName = enrollment_json_object["pharmacyName"]
                #insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
                insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
                insights_model_invoke_timestamp = datetime.now()
                insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
                insights_model_invoke_after_timestamp = datetime.now()
                print("Generating Insights successfully", insights)
                sns_data = sns_data_postprocessing(event,insights)
                print("sns_data : ",sns_data)
                sns_publisher(sns_data)
                insights_redis_data.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
                print(f'insights_redis_data:{insights_redis_data}')
                redis_client.set(redis_key,str(insights_redis_data))
                end_timestamp = datetime.now()
                duration_till_entity_model_invoking = entity_extraction_model_invoke_timestamp - start_timestamp
                duration_of_response_of_entity_model = entity_extraction_model_invoke_after_timestamp - entity_extraction_model_invoke_timestamp
                duration_between_respone_of_entityModel_and_insightsModel = insights_model_invoke_timestamp - entity_extraction_model_invoke_after_timestamp 
                duration_of_response_of_insights_model = insights_model_invoke_after_timestamp - insights_model_invoke_timestamp
                duration_till_end = end_timestamp - insights_model_invoke_after_timestamp
                print(f'insights_data: handler: END: duration_start_to_entity_model::{duration_till_entity_model_invoking}:: duration_entity_model::{duration_of_response_of_entity_model}:: duration_entity_model_to_insights_model::{duration_between_respone_of_entityModel_and_insightsModel}:: duration_insights_model::{duration_of_response_of_insights_model}:: duration_insight_model_to_end::{duration_till_end}:: output: {json.dumps(sns_data)}')
                return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
        else:
            new_value_list = []
            redis_client.set(redis_key,str([])) 
            insurance_provider = enrollment_json_object["insuranceProvider"]
            pharmacyName = enrollment_json_object["pharmacyName"]
            insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
            insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
            insights_model_invoke_timestamp = datetime.now()
            insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
            insights_model_invoke_after_timestamp = datetime.now()
            print("Generating Insights successfully", insights)
            sns_data = sns_data_postprocessing(event,insights)
            print("sns_data : ",sns_data)
            sns_publisher(sns_data)
            new_value_list.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
            print(f'new_value : {new_value_list}')
            redis_client.set(redis_key,str(new_value_list))
            print('Insights Lambda Triggered.')
            end_timestamp = datetime.now()
            duration_till_entity_model_invoking = entity_extraction_model_invoke_timestamp - start_timestamp
            duration_of_response_of_entity_model = entity_extraction_model_invoke_after_timestamp - entity_extraction_model_invoke_timestamp
            duration_between_respone_of_entityModel_and_insightsModel = insights_model_invoke_timestamp - entity_extraction_model_invoke_after_timestamp 
            duration_of_response_of_insights_model = insights_model_invoke_after_timestamp - insights_model_invoke_timestamp
            duration_till_end = end_timestamp - insights_model_invoke_after_timestamp
            print(f'insights_data: handler: END: duration_start_to_entity_model::{duration_till_entity_model_invoking}:: duration_entity_model::{duration_of_response_of_entity_model}:: duration_entity_model_to_insights_model::{duration_between_respone_of_entityModel_and_insightsModel}:: duration_insights_model::{duration_of_response_of_insights_model}:: duration_insight_model_to_end::{duration_till_end}:: output: {json.dumps(sns_data)}')
            return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
    else:
        #sns_data = sns_data_postprocessing(event," ")
        print("No Insigts Generated.")
        return {"statusCode": 200,"body": "No Insigts Generated."}
    #SQS_Publisher(sqs_data)
    #print("sns_data : ",sns_data)

ModuleNotFoundError: No module named 'redis'